In [1]:
import multiprocessing
import os
import sys
import time
sys.path.append("..")
from sklearn.model_selection import train_test_split

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [2]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 17.61245822906494 to run!


## Load pickled multiview action data

In [3]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [4]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Perform basic verification that skeleton data is formatted correctly

In [5]:
for action in X:
    for frame_num, pose in enumerate(action):
        assert pose.shape == (20, 10), f"{pose.shape} =/= (20, 10)"
        for i, descriptor in enumerate(pose):
            if i == 0:
                assert list(descriptor)[:-1] == [0, 0, 0, 0, 0, 0, 0, 0, 0]
            assert descriptor[-1] == frame_num, f"{descriptor[-1]} =/= {frame_num}"

## Create train/test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Pickle Action Classifiers

In [7]:
def pickle_action_classifiers(n_training_neighbors, cache_path):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=20,
        n_training_neighbors=n_training_neighbors,
        alpha=0.75,
        beta=0.6,
        kappa=80
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=0.5,
        n=80
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=cache_path,
        verbose=True
    )

cache_workers = []
for n_training_neighbors in [2000, 5000, 10000, 20000, 30000, 40000]:
    cache_path = f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    if not os.path.exists(cache_path):
        worker = multiprocessing.Process(
            target=
                pickle_action_classifiers,
            args=(
                n_training_neighbors,
                cache_path
            )
        )
        cache_workers.append(worker)

for cache_worker in cache_workers:
    cache_worker.start()
    
for cache_worker in cache_workers:
    cache_worker.join()

## Train Action Classifier with normalized training data

In [ ]:
def score(return_dict, n_neighbors, n_training_neighbors, alpha, beta, kappa, theta, n):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=n_neighbors,
        n_training_neighbors=n_training_neighbors,
        alpha=alpha,
        beta=beta,
        kappa=kappa
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=theta,
        n=n
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    )
    result = action_classifier.predict_all(X_test)
    return_dict[(n_neighbors, n_training_neighbors, alpha, beta, kappa, theta, n)] = (action_classifier, result)

manager = multiprocessing.Manager()
return_dict = manager.dict()
workers = []

for n_neighbors in [30, 40, 50]:
    for n_training_neighbors in [2000, 5000, 20000, 40000]:
        for alpha in [0.75]:
            for beta in [0.6]:
                for kappa in [30, 40, 50]:
                    for theta in [0.6, 0.7, 0.8, 0.9]:
                        for n in [40, 50, 60]:
                            worker = multiprocessing.Process(
                                target=
                                    score,
                                args=(
                                    return_dict,
                                    n_neighbors,
                                    n_training_neighbors,
                                    alpha,
                                    beta,
                                    kappa,
                                    theta,
                                    n
                                )
                            )
                            workers.append(worker)

num_cpu_cores = os.getenv("NUM_CPU_CORES")
if num_cpu_cores is None:
    num_cpu_cores = input("Enter the number of CPU cores this computer has (ex. '12'): ")
num_cpu_cores = max(1, int(num_cpu_cores) - 4)

num_workers_executed = num_cpu_cores
while num_workers_executed < len(workers):
    for i in range(num_cpu_cores):
        workers[num_workers_executed - i].start()
    for i in range(num_cpu_cores):
        os.system('clear')
        print(f"Tuning {100*round((num_workers_executed-(num_cpu_cores - i))/len(workers), 3)}% complete")
        workers[num_workers_executed - i].join()
    num_workers_executed += num_cpu_cores


print("# Correctly predicted: (higher is better)")
best_score = sys.maxsize
best_hpc = None
best_action_classifier = None
for hyper_parameter_combo, (action_classifier, result) in return_dict.items():
    if result > best_score:
        best_hpc = hyper_parameter_combo
        best_action_classifier = action_classifier
        best_score = result


Enter the number of CPU cores this computer has (ex. '12'): 24
Tuning 0.0% complete
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.96000000e+02]
 [-3.59248123e-03  5.11439201e-02  4.84984965e-02 -9.90530640e-06
  -1.27713847e-04  1.33721636e-04 -1.79522919e-05 -2.79629745e-04
   2.91562464e-04  3.96000000e+02]
 [-2.83667093e-02  3.65310762e-01  7.95843334e-02  2.31955434e-04
  -3.54949598e-04  2.60947546e-03  4.36622229e-04 -4.03914559e-04
   1.71846525e-03  3.96000000e+02]
 [-7.17374782e-03  5.65857011e-01  4.60539562e-02 -3.19372270e-04
  -8.04468860e-04 -4.11026863e-04 -4.89940023e-04 -6.12939536e-04
   1.59866221e-04  3.96000000e+02]
 [-1.82324674e-01  2.40243581e-01  8.29466516e-02 -4.43409862e-04
   4.07343761e-04 -1.73869275e-04 -4.45506187e-04  7.10910042e-04
   8.71264116e-05  3.96000000e+02]
 [-7.27256775e-02  1.03552697e-01 -7.96046693e-02 -4.43409862e-04
   4.07

   1.30364403e-04  3.98000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.99000000e+02]
 [-3.61433457e-03  5.06006840e-02  4.90634081e-02  2.80969860e-06
  -5.29427489e-04  5.45694612e-04  2.13319005e-05 -7.95353924e-05
   7.39253712e-05  3.99000000e+02]
 [-2.76021010e-02  3.64337132e-01  8.47694845e-02  4.99748911e-04
  -8.40344077e-04  3.61835280e-03  6.30227796e-05 -5.28709193e-05
  -3.96320339e-04  3.99000000e+02]
 [-8.35828241e-03  5.64046193e-01  4.55496871e-02 -1.28947517e-03
  -1.49994058e-03 -6.13758478e-04 -2.63179506e-04 -1.06884553e-04
  -3.40643052e-04  3.99000000e+02]
 [-1.83300348e-01  2.41415497e-01  8.26511410e-02 -5.84962390e-04
   6.03643453e-04 -5.07511205e-04  4.35408752e-04 -3.75779018e-04
  -3.50195101e-04  3.99000000e+02]
 [-7.44591802e-02  1.04507129e-01 -8.02260171e-02 -1.64434718e-03
   4.69837772e-04 -1.10228568e-03 -1.29828469e-04  2.33

  -3.29736053e-04  4.01000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.02000000e+02]
 [-3.60164416e-03  4.99341929e-02  4.97424888e-02  3.33931089e-06
  -3.15444859e-04  3.16786287e-04 -2.26766994e-05  1.70633613e-04
  -1.76692864e-04  4.02000000e+02]
 [-2.67443715e-02  3.63292655e-01  8.91351808e-02  2.64624600e-04
  -4.90191974e-04  1.86730259e-03 -5.63319545e-04  2.36162747e-04
  -1.10571181e-03  4.02000000e+02]
 [-9.81032178e-03  5.62098764e-01  4.46200697e-02 -3.41567981e-04
  -9.66376383e-04 -4.99799767e-04  5.71434800e-04  3.52225917e-04
   3.02954360e-05  4.02000000e+02]
 [-1.83626053e-01  2.42090122e-01  8.17701483e-02 -1.41795936e-04
   1.84132657e-04 -6.32288716e-04  6.48572049e-05 -5.65075526e-04
  -1.61322652e-04  4.02000000e+02]
 [-7.65380220e-02  1.05828221e-01 -8.28023815e-02 -1.01973791e-03
   1.16160120e-03 -2.01285332e-03  2.60225178e-04 -2.30

   2.15286943e-04  4.04000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.05000000e+02]
 [-3.68009875e-03  4.96502120e-02  5.00202356e-02 -2.91975937e-04
  -3.57195636e-04  3.31246962e-04 -5.07996731e-04 -4.48679023e-04
   3.98465318e-04  4.05000000e+02]
 [-2.67498651e-02  3.62751873e-01  9.14452518e-02 -2.37187763e-04
  -5.04170264e-04  1.47705877e-03 -4.89381976e-04 -3.31335803e-04
  -5.05993312e-04  4.05000000e+02]
 [-1.00364523e-02  5.60925196e-01  4.41142737e-02 -2.66099390e-04
  -7.81848792e-04  2.99224517e-04 -4.33952744e-04 -9.75060587e-05
   5.33780662e-04  4.05000000e+02]
 [-1.83526300e-01  2.41675243e-01  8.05935669e-02  2.31077555e-04
  -8.84821072e-04 -1.03464848e-03 -1.07275640e-04 -7.74955124e-04
  -5.24681698e-04  4.05000000e+02]
 [-7.78188659e-02  1.06920575e-01 -8.60989270e-02 -3.04251596e-04
   2.58034748e-04 -2.29846568e-03  6.91330267e-04 -4.38

   8.99500154e-06  4.07000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.08000000e+02]
 [-4.51903476e-03  4.90262862e-02  5.05639420e-02 -5.13885590e-04
  -2.13688877e-05 -2.52389751e-05 -1.75873660e-05  2.89290579e-04
  -2.91652029e-04  4.08000000e+02]
 [-2.74811563e-02  3.62043422e-01  9.26817482e-02 -2.17216620e-04
  -4.41492514e-05  3.05293712e-04  5.73191095e-04  3.37560005e-04
  -3.35234048e-04  4.08000000e+02]
 [-1.07918365e-02  5.59931071e-01  4.41619497e-02 -2.38523315e-04
  -2.31877022e-04 -4.68799650e-04  4.81740418e-04  3.76586773e-04
  -1.95988572e-04  4.08000000e+02]
 [-1.83353599e-01  2.40175188e-01  7.82569464e-02  2.71935222e-04
  -4.11712253e-04 -1.89265100e-03  1.55196545e-04  8.62922290e-04
   7.72205727e-05  4.08000000e+02]
 [-7.79997115e-02  1.07571148e-01 -9.03728155e-02 -3.37164611e-04
   1.49511587e-03 -3.76892620e-03 -8.24673270e-04  1.54

...

  -5.20082908e-05  5.35000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.36000000e+02]
 [-2.67787840e-03  5.14635956e-02  4.82186472e-02 -2.75044059e-04
  -2.74866064e-04  2.77211717e-04 -5.14714834e-04 -1.90893401e-04
   1.69480762e-04  5.36000000e+02]
 [-3.11562854e-02  3.65267017e-01  7.97985327e-02 -5.68155673e-04
  -3.88797509e-04  1.14434609e-03 -3.16322518e-04 -2.25414854e-04
   6.62972371e-04  5.36000000e+02]
 [-1.24439531e-02  5.67423794e-01  5.58253865e-02 -5.65696959e-04
  -4.95845938e-04  2.41688089e-04  6.38597570e-04 -3.28017041e-04
   5.80332264e-04  5.36000000e+02]
 [-1.88487578e-01  2.45406917e-01  9.51797164e-02 -6.84220896e-04
  -3.64555717e-04  1.43590524e-04 -2.20320610e-04 -4.21211244e-04
   4.01376349e-05  5.36000000e+02]
 [-9.15367892e-02  1.10412148e-01 -7.65649383e-02 -7.91116149e-04
  -2.15714225e-04 -3.38652319e-05  4.78169610e-05 -7.01

   3.20253134e-04  5.38000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.39000000e+02]
 [-3.44606385e-03  5.10096515e-02  4.86502903e-02 -2.47114091e-04
  -3.13043446e-04  3.10851167e-04  5.26596934e-04 -3.88755627e-05
   7.13118753e-05  5.39000000e+02]
 [-3.20251224e-02  3.64597565e-01  8.22174472e-02 -6.29769892e-04
  -5.46201497e-04  2.15743546e-03 -3.99680900e-05 -2.00673593e-04
   9.72145431e-04  5.39000000e+02]
 [-1.28021216e-02  5.66494040e-01  5.65169504e-02 -5.53708660e-04
  -8.30913715e-04 -5.08656582e-06 -9.62811140e-05 -4.62055411e-04
  -8.84186303e-04  5.39000000e+02]
 [-1.89769661e-01  2.45007890e-01  9.52991150e-02 -1.12259800e-03
  -1.27708851e-04  2.46842260e-05 -3.38306529e-04  1.25643250e-04
  -1.10381116e-04  5.39000000e+02]
 [-9.28383636e-02  1.10040260e-01 -7.64778685e-02 -1.12259800e-03
  -1.27708851e-04  2.46842260e-05 -4.13655209e-04  1.37

  -2.96317238e-03  5.41000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.42000000e+02]
 [-3.28187716e-03  5.03958123e-02  4.92970794e-02  5.55712277e-04
  -9.88139313e-04  1.03782886e-03  5.12290964e-04 -1.18968992e-03
   1.18560250e-03  5.42000000e+02]
 [-3.22389806e-02  3.63444326e-01  8.72840532e-02  1.40418509e-03
  -1.41637286e-03  5.19863577e-03  2.37650019e-03 -1.17471393e-03
   2.04123947e-03  5.42000000e+02]
 [-1.25031948e-02  5.64441075e-01  5.56133777e-02  1.45583741e-03
  -2.37926535e-03 -8.35389944e-04  9.29777701e-04 -1.48900651e-03
   2.07197387e-04  5.42000000e+02]
 [-1.91289268e-01  2.45119726e-01  9.49296397e-02 -2.01524833e-04
   3.89470736e-04 -5.78084886e-04  1.19853212e-03  5.83803608e-04
  -1.97163151e-04  5.42000000e+02]
 [-9.48269572e-02  1.10201950e-01 -7.71502434e-02 -1.43777810e-03
   4.30181433e-04 -1.30093461e-03 -1.56783762e-04  4.11

   2.22696496e-03  5.44000000e+02]]
Cur X:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.45000000e+02]
 [-2.43829277e-03  4.79623296e-02  5.17144111e-02 -1.86916916e-04
  -2.90781164e-04  2.67052880e-04  1.19797024e-04  1.78349669e-03
  -1.71107000e-03  5.45000000e+02]
 [-2.81643829e-02  3.60451146e-01  9.60968807e-02  5.01130569e-06
  -2.69660259e-04  2.33630327e-04 -2.96310600e-03  2.09176662e-03
  -5.94782858e-03  5.45000000e+02]
 [-1.02878333e-02  5.59872432e-01  5.48197007e-02 -2.60760762e-04
  -2.58736562e-04  1.61122828e-04 -9.67244329e-04  3.10586659e-03
   3.53282925e-04  5.45000000e+02]
 [-1.90632082e-01  2.46629654e-01  9.38442051e-02 -8.77630064e-05
  -1.40078621e-04  2.32372252e-04 -4.66626999e-04 -1.33137652e-03
   7.67306626e-04  5.45000000e+02]
 [-9.70597711e-02  1.11482792e-01 -7.96459041e-02 -2.09809243e-04
  -1.22451532e-04  1.51908627e-04  1.90232644e-03 -1.30

   3.35115064e-04  5.47000000e+02]]
Tuning 0.2% complete
Tuning 0.5% complete
Tuning 0.7000000000000001% complete
Tuning 0.8999999999999999% complete
Tuning 1.2% complete
Tuning 1.4000000000000001% complete
Tuning 1.6% complete
Tuning 1.9% complete
Tuning 2.1% complete
Tuning 2.3% complete
Tuning 2.5% complete
Tuning 2.8000000000000003% complete
Tuning 3.0% complete
Tuning 3.2% complete
Tuning 3.5000000000000004% complete
Tuning 3.6999999999999997% complete
Tuning 3.9% complete
Tuning 4.2% complete
Tuning 4.3999999999999995% complete
Tuning 4.6% complete


## Print best action classifier and its score

In [ ]:
print("The best hyper parameter combo was:")
print(f"    n_neighbors: {best_hpc[0]}")
print(f"    n_training_neighbors: {best_hpc[1]}")
print(f"    alpha: {best_hpc[2]}")
print(f"    beta: {best_hpc[3]}")
print(f"    kappa: {best_hpc[4]}")
print(f"    theta: {best_hpc[5]}")
print(f"    n: {best_hpc[6]}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
